In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
import keras
import seaborn as sns
from google.colab import drive
from functools import partial
import flwr as fl
from tensorflow.keras.models import Model


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install -q flwr["simulation"] tensorflow
!pip install -q flwr_datasets["vision"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/data/Prepared/ciciot2023-reduced.csv')

In [ ]:
data

,Unnamed: 0,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0,0.000000,0.0,1.0,64.0,30.337559,30.337559,0.0,0.0,0.0,...,0.000000,42.0,8.314949e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
1,1,0.000000,0.0,1.0,64.0,0.689640,0.689640,0.0,0.0,0.0,...,0.000000,42.0,8.312898e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
2,2,0.000000,0.0,1.0,64.0,1.350653,1.350653,0.0,0.0,0.0,...,0.000000,42.0,8.314964e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
3,3,0.000000,0.0,1.0,64.0,3.637768,3.637768,0.0,0.0,0.0,...,0.000000,42.0,8.312906e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
4,4,0.000000,0.0,1.0,64.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,42.0,8.312889e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177093,1177093,21.931409,3877.2,3.1,57.1,35.848699,35.848699,0.0,0.0,0.0,...,251.519584,147.8,3.352358e-02,5.5,20.309082,355.702407,72476.897264,0.9,38.50,Uploading_Attack
1177094,1177094,623.726814,86935.3,5.5,70.4,0.214131,0.214131,0.0,0.0,0.0,...,437.244141,684.1,1.630549e-02,5.5,22.382339,618.356594,568501.573782,0.9,38.50,Uploading_Attack
1177095,1177095,82.325677,7351.8,13.2,71.2,28.355732,28.355732,0.0,0.0,0.0,...,47.763924,157.1,1.115448e-02,5.5,17.417450,67.548389,2551.505968,0.9,38.50,Uploading_Attack
1177096,1177096,45.667144,2508.2,10.4,111.6,129097.868387,129097.868387,0.0,0.0,0.0,...,45.064730,109.2,4.131398e-02,5.5,14.717295,63.731153,2333.652536,0.9,38.50,Uploading_Attack


In [ ]:
data.drop(columns='Unnamed: 0', inplace=True)
data

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,0.0,1.0,64.0,30.337559,30.337559,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.314949e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
1,0.000000,0.0,1.0,64.0,0.689640,0.689640,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312898e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
2,0.000000,0.0,1.0,64.0,1.350653,1.350653,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.314964e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
3,0.000000,0.0,1.0,64.0,3.637768,3.637768,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312906e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
4,0.000000,0.0,1.0,64.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312889e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177093,21.931409,3877.2,3.1,57.1,35.848699,35.848699,0.0,0.0,0.0,0.0,...,251.519584,147.8,3.352358e-02,5.5,20.309082,355.702407,72476.897264,0.9,38.50,Uploading_Attack
1177094,623.726814,86935.3,5.5,70.4,0.214131,0.214131,0.0,0.0,0.0,0.0,...,437.244141,684.1,1.630549e-02,5.5,22.382339,618.356594,568501.573782,0.9,38.50,Uploading_Attack
1177095,82.325677,7351.8,13.2,71.2,28.355732,28.355732,0.0,0.0,0.0,0.0,...,47.763924,157.1,1.115448e-02,5.5,17.417450,67.548389,2551.505968,0.9,38.50,Uploading_Attack
1177096,45.667144,2508.2,10.4,111.6,129097.868387,129097.868387,0.0,0.0,0.0,0.0,...,45.064730,109.2,4.131398e-02,5.5,14.717295,63.731153,2333.652536,0.9,38.50,Uploading_Attack


In [ ]:
## Checking to see if some of the columns contain all zeros
columns = data.columns

result = np.sum(data[columns])
result[result == 0]

Telnet    0.0
SMTP      0.0
DHCP      0.0
dtype: object

In [ ]:
### Droping the columns all zeros as entry
data.drop(columns=['DHCP', 'Telnet','SMTP'], inplace=True)
data

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,0.0,1.0,64.0,30.337559,30.337559,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.314949e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
1,0.000000,0.0,1.0,64.0,0.689640,0.689640,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312898e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
2,0.000000,0.0,1.0,64.0,1.350653,1.350653,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.314964e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
3,0.000000,0.0,1.0,64.0,3.637768,3.637768,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312906e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
4,0.000000,0.0,1.0,64.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312889e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,DDoS-ICMP_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177093,21.931409,3877.2,3.1,57.1,35.848699,35.848699,0.0,0.0,0.0,0.0,...,251.519584,147.8,3.352358e-02,5.5,20.309082,355.702407,72476.897264,0.9,38.50,Uploading_Attack
1177094,623.726814,86935.3,5.5,70.4,0.214131,0.214131,0.0,0.0,0.0,0.0,...,437.244141,684.1,1.630549e-02,5.5,22.382339,618.356594,568501.573782,0.9,38.50,Uploading_Attack
1177095,82.325677,7351.8,13.2,71.2,28.355732,28.355732,0.0,0.0,0.0,0.0,...,47.763924,157.1,1.115448e-02,5.5,17.417450,67.548389,2551.505968,0.9,38.50,Uploading_Attack
1177096,45.667144,2508.2,10.4,111.6,129097.868387,129097.868387,0.0,0.0,0.0,0.0,...,45.064730,109.2,4.131398e-02,5.5,14.717295,63.731153,2333.652536,0.9,38.50,Uploading_Attack


In [ ]:
# Creating a dictionary of attack types for 33 attack classes + 1 for benign traffic
dict_34_classes = {'BenignTraffic': 0 ,
                    'DDoS-RSTFINFlood' :1, 'DDoS-PSHACK_Flood':2,  'DDoS-SYN_Flood':3, 'DDoS-UDP_Flood':4, 'DDoS-TCP_Flood':5,
                    'DDoS-ICMP_Flood':6, 'DDoS-SynonymousIP_Flood':7, 'DDoS-ACK_Fragmentation':8, 'DDoS-UDP_Fragmentation':9, 'DDoS-ICMP_Fragmentation':10,
                    'DDoS-SlowLoris':11, 'DDoS-HTTP_Flood':12, 'DoS-UDP_Flood':13, 'DoS-SYN_Flood':14, 'DoS-TCP_Flood':15, 'DoS-HTTP_Flood':16,                 # DDoS and DoS
                    'Mirai-greeth_flood': 17, 'Mirai-greip_flood': 18, 'Mirai-udpplain': 19,                                                                    # Mirai
                    'Recon-PingSweep': 20, 'Recon-OSScan': 21, 'Recon-PortScan': 22, 'VulnerabilityScan': 23, 'Recon-HostDiscovery': 24,                        # Reconnaissance
                    'DNS_Spoofing': 25, 'MITM-ArpSpoofing': 26,                                                                                                 # Spoofing
                    'BrowserHijacking': 27, 'Backdoor_Malware': 28, 'XSS': 29, 'Uploading_Attack': 30, 'SqlInjection': 31, 'CommandInjection': 32,              # Web
                    'DictionaryBruteForce': 33}                                                                                                                 # Brute Force

dict_7_classes = {  0: 0 ,
                    1 :1, 2:1,  3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1,                                                    # DDoS and DoS
                    17: 2, 18: 2, 19: 2,                                                                                                                        # Mirai
                    20: 3, 21: 3, 22: 3, 23: 3, 24: 3,                                                                                                          # Reconnaissance
                    25: 4, 26: 4,                                                                                                                               # Spoofing
                    27: 5, 28: 5, 29: 5, 30: 5, 31: 5, 32: 5,                                                                                                   # Web
                    33: 6}                                                                                                                                      # Brute Force

dict_2_classes = {  0: 0 ,
                    1 :1, 2:1,  3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1,                                                    # DDoS and DoS
                    17: 1, 18: 1, 19: 1,                                                                                                                        # Mirai
                    20: 1, 21: 1, 22: 1, 23: 1, 24: 1,                                                                                                          # Reconnaissance
                    25: 1, 26: 1,                                                                                                                               # Spoofing
                    27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1,                                                                                                   # Web
                    33: 1}                                                                                                                                      # Brute Force

# Map y column to the dict_34_classes values
data['label'] = data['label'].map(dict_34_classes)
data['label'] = data['label'].map(dict_2_classes)

In [ ]:
## Shuffle the dataset before the split
data.sample(frac=1).reset_index(drop=True)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,3.44,46.23,63.36,64.254918,64.254918,0.0,0.0,0.0,0.0,...,14.068555,570.48,8.357376e+07,9.5,33.890677,19.952858,1600.297628,0.14,141.55,1
1,8.644729,928674.20,6.00,114.40,97.573832,97.573832,0.0,0.0,0.0,0.0,...,174.442905,1079.60,7.900000e-04,5.5,53.212215,246.699522,103202.812395,0.30,38.50,0
2,46.621915,59960.00,7.10,97.10,15.040051,15.040051,0.0,0.0,0.0,0.0,...,47.697179,89.60,1.665182e+08,13.5,14.314998,67.539587,2281.608511,1.00,244.60,0
3,0.055252,28911.50,16.83,63.36,11944.329252,11944.329252,0.0,0.0,0.0,0.0,...,0.067030,50.10,8.349523e+07,9.5,10.001604,0.095552,0.152192,0.03,141.55,1
4,10.333232,529628.70,12.60,171.00,36.376267,36.376267,0.0,0.0,0.0,0.0,...,100.593775,146.40,1.665244e+08,13.5,15.552927,142.458625,10233.652039,1.00,244.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177093,0.126210,60.54,6.00,63.82,1.918367,1.918367,0.0,0.0,1.0,0.0,...,1.196540,54.24,8.336183e+07,9.5,10.432530,1.693979,8.822682,0.17,141.55,1
1177094,0.252003,19718.20,6.00,64.00,202.888844,202.888844,0.0,0.0,0.0,0.0,...,9.195120,76.40,1.665176e+08,13.5,12.213453,13.011180,84.850337,1.00,244.60,0
1177095,63.856496,3227821.70,7.10,234.30,52.649414,52.649414,0.0,0.0,0.0,0.0,...,8.580000,94.60,1.349998e-03,5.5,11.715713,12.133952,736.164000,0.10,38.50,0
1177096,69.279470,97197.40,7.60,94.60,8.235510,8.235510,0.0,0.0,0.0,0.0,...,55.431193,139.40,3.181670e-02,5.5,16.974180,78.391545,3421.490719,0.90,38.50,0


In [ ]:
### Dividing the dataset into train, val and test set
X = data.drop('label', axis=1)
y = data['label']


In [ ]:
### splitting X and y into train, validation and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"X train {X_train.shape}")
# print(f"X val {X_val.shape}")
print(f"X test {X_test.shape}")
print(f"Y train {y_train.shape}")
# print(f"Y val {y_val.shape}")
print(f"Y test {y_test.shape}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


X train (1059388, 43)
X test (117710, 43)
Y train (1059388,)
Y test (117710,)


In [ ]:
### Scaling the splits
scalar = StandardScaler()
scalar.fit(X_train.reset_index(drop=True))

X_train_scaled = scalar.transform(X_train.reset_index(drop=True))
# X_val_scaled = scalar.transform(X_val.reset_index(drop=True))
X_test_scaled = scalar.transform(X_test.reset_index(drop=True))
y_test_scaled = y_test.to_numpy()
# y_val_scaled = y_val.to_numpy()
y_train_scaled = y_train.to_numpy()

In [ ]:

NUM_CLIENTS = 5
partition_size = X_train_scaled.shape[0] // NUM_CLIENTS
client_id_to_indices = {}
beg_ids = [i * partition_size for i in range(NUM_CLIENTS)]
end_ids = [i * partition_size for i in range(1, NUM_CLIENTS + 1)]
for client_id, (beg_id, end_id) in enumerate(zip(beg_ids, end_ids)):
    client_id_to_indices[client_id] = [beg_id, end_id]

x_split = np.array_split(X_train_scaled, NUM_CLIENTS)
y_split = np.array_split(y_train_scaled, NUM_CLIENTS)
num_data_in_split = x_split[0].shape[0]
train_split = 0.8
X_trains, y_trains, X_vals, y_vals = {}, {}, {}, {}
for idx, (client_x, client_y) in enumerate(zip(x_split, y_split)):
    train_end_idx = int(0.8 * num_data_in_split)
    X_trains[str(idx)] = client_x[:train_end_idx]
    y_trains[str(idx)] = client_y[:train_end_idx]
    X_vals[str(idx)] = client_x[train_end_idx:]
    y_vals[str(idx)] = client_y[train_end_idx:]

In [ ]:
print(f"Partition size: {partition_size}")
print(beg_ids)
end_ids

Partition size: 211877
[0, 211877, 423754, 635631, 847508]


[211877, 423754, 635631, 847508, 1059385]

In [ ]:
class ANN(Model):
    def __init__(self):
        super(ANN, self).__init__()
        self.dense1 = Dense(43, activation='relu', input_dim=(43))
        # self.dropout = Dropout(0.5)
        self.dense2 = Dense(128, activation='relu')
        # self.dense3 = Dense(128, activation='relu')
        self.dense4 = Dense(128, activation='relu')
        self.dense5 = Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        # x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

In [ ]:
# ## Defining flower client instance
# class FlowerClient(fl.client.NumPyClient):
#     def __init__(self, model, X_train, y_train, X_val, y_val):
#         self.model = model
#         self.model.build(input_shape=(None, 43))
#         self.X_train = X_train
#         self.y_train = y_train
#         self.X_val = X_val
#         self.y_val = y_val

#     def get_parameters(self, config):
#         return self.model.get_weights()

#     def fit(self, parameters, config):
#         self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#         self.model.set_weights(parameters)j
#         history = self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
#         return self.model.get_weights(), len(self.X_train), {"train_loss": history.history['loss'][0], "train_accuracy": history.history['accuracy'][0]}

#     def evaluate(self, parameters, config):
#         self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#         # self.model.build(input_shape=(32, 28, 28, 1))
#         self.model.set_weights(parameters)
#         loss, accuracy = self.model.evaluate(self.X_val, self.y_val, batch_size=32, verbose=0)
#         return loss, len(self.X_val), {"val_loss": loss, "val_accuracy": accuracy}




@tf.autograph.experimental.do_not_convert
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, X_train, y_train, X_val, y_val):
        self.model = model
        self.model.build(input_shape=(None, 43))
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
        self.model.set_weights(parameters)
        history = self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=1)
        train_loss = history.history['loss'][0]
        train_accuracy = history.history['accuracy'][0]
        return self.model.get_weights(), len(self.X_train), {"train_loss": train_loss, "train_accuracy": train_accuracy}

    def evaluate(self, parameters, config):
        self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
        self.model.set_weights(parameters)
        val_loss, val_accuracy = self.model.evaluate(self.X_val, self.y_val, batch_size=32, verbose=1)
        return val_loss, len(self.X_val), {"val_loss": val_loss, "val_accuracy": val_accuracy}


# class FlowerClient(fl.client.NumPyClient):
#     def __init__(self, model, X_train, y_train, X_val, y_val):
#         self.model = model
#         self.model.build(input_shape=(None, 43))
#         self.X_train = X_train
#         self.y_train = y_train
#         self.X_val = X_val
#         self.y_val = y_val

#     def get_parameters(self, config):
#         return self.model.get_weights()

#     def fit(self, parameters, config):
#         self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#         self.model.set_weights(parameters)
#         history = self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0, callbacks=[tf.keras.callbacks.History()])
#         train_loss = history.history['loss'][0]
#         train_accuracy = history.history['accuracy'][0]
#         return self.model.get_weights(), len(self.X_train), {"train_loss": train_loss, "train_accuracy": train_accuracy}

#     def evaluate(self, parameters, config):
#         self.model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#         self.model.set_weights(parameters)
#         val_loss, val_accuracy = self.model.evaluate(self.X_val, self.y_val, batch_size=32, verbose=0)
#         return val_loss, len(self.X_val), {"val_loss": val_loss, "val_accuracy": val_accuracy}


In [ ]:
## Defining the client function
def create_client(
    cid, model_class, X_trains, y_trains, X_vals, y_vals
) -> FlowerClient:
    """Create a Flower client representing a single organization."""
    model = model_class()
    # Create a  single Flower client representing a single organization
    return FlowerClient(model, X_trains[cid], y_trains[cid], X_vals[cid], y_vals[cid]).to_client()


client_fnc = partial(
    create_client,
    model_class=ANN,
    X_trains=X_trains,
    y_trains=y_trains,
    X_vals=X_vals,
    y_vals=y_vals,
)

In [ ]:
## Defining the strategy
# def weighted_average(metrics):
#     accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
#     examples = [num_examples for num_examples, _ in metrics]
#     return {"accuracy": int(sum(accuracies)) / int(sum(examples))}
def weighted_average(metrics):
    total_samples = sum(num_samples for num_samples, _ in metrics)
    global_val_loss = sum(num_samples * m["val_loss"] for num_samples, m in metrics) / total_samples
    global_val_accuracy = sum(num_samples * m["val_accuracy"] for num_samples, m in metrics) / total_samples
    return {"global_val_loss": global_val_loss, "global_val_accuracy": global_val_accuracy}

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,  # Sample 100% of available clients for evaluation
    min_fit_clients=2,  # Never sample less than 2 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=5,  # Wait until all 5 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
)

In [ ]:
# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fnc,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 0},
    ray_init_args={
        "num_cpus": 1,
        "num_gpus": 0,
        "_system_config": {"automatic_object_spilling_enabled": False},
    },
)


INFO flwr 2024-03-25 18:26:23,814 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2024-03-25 18:26:29,855	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-25 18:26:31,172 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 1.0, 'object_store_memory': 3870083481.0, 'memory': 7740166964.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 1.0, 'object_store_memory': 3870083481.0, 'memory': 7740166964.0}
INFO flwr 2024-03-25 18:26:31,180 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-25 18:

5297/5297 [==============================] - 11s 2ms/step - loss: 0.0617 - accuracy: 0.9847


(DefaultActor pid=134993) WARNING:tensorflow:AutoGraph could not transform <bound method ANN.call of <__main__.ANN object at 0x7dae285fbc10>> and will run it as-is.
(DefaultActor pid=134993) Cause: Unable to locate the source code of <bound method ANN.call of <__main__.ANN object at 0x7dae285fbc10>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(DefaultActor pid=134993) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


5297/5297 [==============================] - 12s 2ms/step - loss: 0.0628 - accuracy: 0.9841


(DefaultActor pid=134993) WARNING:tensorflow:AutoGraph could not transform <bound method ANN.call of <__main__.ANN object at 0x7dae288ed7b0>> and will run it as-is.
(DefaultActor pid=134993) Cause: Unable to locate the source code of <bound method ANN.call of <__main__.ANN object at 0x7dae288ed7b0>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(DefaultActor pid=134993) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

1287/1325 [============================>.] - ETA: 0s - loss: 0.0418 - accuracy: 0.9888


(DefaultActor pid=134993) WARNING:tensorflow:AutoGraph could not transform <bound method ANN.call of <__main__.ANN object at 0x7dae19800d30>> and will run it as-is.
(DefaultActor pid=134993) Cause: Unable to locate the source code of <bound method ANN.call of <__main__.ANN object at 0x7dae19800d30>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(DefaultActor pid=134993) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


1152/1325 [=========================>....] - ETA: 0s - loss: 0.0414 - accuracy: 0.9886


DEBUG flwr 2024-03-25 18:33:53,093 | server.py:187 | evaluate_round 5 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 5 results and 0 failures
INFO flwr 2024-03-25 18:33:53,097 | server.py:153 | FL finished in 435.1463156659993
INFO:flwr:FL finished in 435.1463156659993
INFO flwr 2024-03-25 18:33:53,115 | app.py:226 | app_fit: losses_distributed [(1, 0.04956280286650101), (2, 0.04581116873179029), (3, 0.04435503950973932), (4, 0.04301970699274802), (5, 0.04277746435744126)]
INFO:flwr:app_fit: losses_distributed [(1, 0.04956280286650101), (2, 0.04581116873179029), (3, 0.04435503950973932), (4, 0.04301970699274802), (5, 0.04277746435744126)]
INFO flwr 2024-03-25 18:33:53,117 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-25 18:33:53,120 | app.py:228 | app_fit: metrics_distributed {'global_val_loss': [(1, 0.04956280286650101), (2, 0.04581116873179029), (3, 0.04435503950973932), (4, 0.043019706992748

1301/1325 [============================>.] - ETA: 0s - loss: 0.0405 - accuracy: 0.9889


In [ ]:
hist

History (loss, distributed):
	round 1: 0.04956280286650101
	round 2: 0.04581116873179029
	round 3: 0.04435503950973932
	round 4: 0.04301970699274802
	round 5: 0.04277746435744126
History (metrics, distributed, evaluate):
{'global_val_loss': [(1, 0.04956280286650101), (2, 0.04581116873179029), (3, 0.04435503950973932), (4, 0.04301970699274802), (5, 0.04277746435744126)], 'global_val_accuracy': [(1, 0.9867282103409234), (2, 0.9878137422751352), (3, 0.9880214069291428), (4, 0.9883659459741733), (5, 0.9883848286810777)]}

In [ ]:
print(hist)

History (loss, distributed):
	round 1: 0.05158547916120722
	round 2: 0.04675355606336818
	round 3: 0.044476861416401635
	round 4: 0.043690164219340306
	round 5: 0.04301932007495626
History (metrics, distributed, evaluate):
{'global_val_loss': [(1, 0.05158547916120722), (2, 0.04675355606336818), (3, 0.044476861416401635), (4, 0.043690164219340306), (5, 0.04301932007495626)], 'global_val_accuracy': [(1, 0.9867990084039532), (2, 0.9874833649817801), (3, 0.9878845410065714), (4, 0.9880733339835074), (5, 0.988323471303375)]}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
